In [5]:
import pandas as pd

def course_recommendation_system(file_path, target_year=114, target_semester=2, target_class='資工二'):
    """
    系級排課與通識推薦系統演算法 (修正版)
    """
    # 1. 讀取資料
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("錯誤：找不到檔案，請確認檔案路徑是否正確。")
        return

    # --- 預處理：計算歷史核心通識統計數據 (111-113學年度) ---
    # 利用過去的平均值來預測 114-2 的熱門程度
    history_mask = (df['學年度'] < target_year) & (df['開課班別(代表)'] == '核心通識')
    ge_history = df[history_mask].groupby('課程名稱').agg({
        '飽和度': 'mean',
        '中籤率': 'mean'
    }).reset_index().rename(columns={'飽和度': '歷史飽和度', '中籤率': '歷史中籤率'})

    # --- Step 1: 列出該系級開設之系必修及選修 ---
    dept_courses = df[
        (df['學年度'] == target_year) & 
        (df['學期'] == target_semester) & 
        (df['開課班別(代表)'] == target_class)
    ].copy()

    if dept_courses.empty:
        print(f"找不到 {target_year}-{target_semester} {target_class} 的開課資料。")
        return

    print(f"\n===== Step 1: {target_year}-{target_semester} {target_class} 開課清單 =====")
    # 這裡顯示給使用者看，讓他們挑選
    print(dept_courses[['課程名稱', '開課班別(代表)', '學分', '教師姓名', '星期', '起始節次', 'cluster標籤', '地點']])
    
    # --- 模擬使用者挑選 (修正：改為動態抓取索引) ---
    # 在實際網頁中，這裡會接收來自前端的使用者勾選 ID
    user_selected_indices = dept_courses.index[:2].tolist()  # 自動選取清單前兩門課作為模擬
    selected_df = dept_courses.loc[user_selected_indices]
    
    print(f"\n[模擬選課] 使用者已選擇：")
    for _, row in selected_df.iterrows():
        print(f" ->  {row['課程名稱']} {row['開課班別(代表)']} (週{row['星期']} 第{row['起始節次']}節起，{row['學分']}學分)")

    # --- Step 2: 動態推薦通識課 (避開衝堂) ---
    
    # 取得 114-2 的核心通識課
    ge_candidates = df[
        (df['學年度'] == target_year) & 
        (df['學期'] == target_semester) & 
        (df['開課班別(代表)'] == '核心通識')
    ].copy()
    
    # 合併歷史熱門度數據
    ge_candidates = ge_candidates.merge(ge_history, on='課程名稱', how='left')
    ge_candidates['歷史飽和度'] = ge_candidates['歷史飽和度'].fillna(0)
    ge_candidates['歷史中籤率'] = ge_candidates['歷史中籤率'].fillna(1.0)

    def check_conflict(ge_row, selected_courses_df):
        """
        衝堂檢測邏輯
        """
        ge_start = ge_row['起始節次']
        ge_end = ge_start + int(ge_row['學分']) - 1
        
        for _, sel in selected_courses_df.iterrows():
            if ge_row['星期'] == sel['星期']:
                sel_start = sel['起始節次']
                sel_end = sel_start + int(sel['學分']) - 1
                # 判定區間是否重疊
                if max(ge_start, sel_start) <= min(ge_end, sel_end):
                    return True
        return False

    # 標記並過濾衝堂課程
    ge_candidates['is_conflict'] = ge_candidates.apply(lambda row: check_conflict(row, selected_df), axis=1)
    recommendations = ge_candidates[ge_candidates['is_conflict'] == False].copy()

    # 排序：依照「歷史飽和度」降序（熱門），再依照「歷史中籤率」降序（易選）
    recommendations = recommendations.sort_values(by=['歷史飽和度', '歷史中籤率'], ascending=[False, False])

    print(f"\n===== Step 2: 推薦核心通識 (已排除衝堂，按歷史熱門度排序) =====")
    if not recommendations.empty:
        # 顯示前 10 筆推薦
        top_10 = recommendations[['課程名稱', '開課班別(代表)', '教師姓名', '星期', '起始節次', '歷史飽和度', '歷史中籤率', '地點']].head(10)
        print(top_10.to_string(index=False))
    else:
        print("抱歉，目前時間排程下沒有不衝堂的通識課。")

# 執行
file_name = 'course_cluster3(1).csv' # 請確保檔案在此路徑
course_recommendation_system(file_name)


===== Step 1: 114-2 資工二 開課清單 =====
              課程名稱 開課班別(代表)   學分 教師姓名  星期  起始節次 cluster標籤        地點
14929      可程式邏輯設計      資工二  3.0  陳伯岳   5     2      全英專業    資工電腦教室
14930       微處理機技術      資工二  2.0  伍朝欽   4     5      一般課程    資工電腦教室
14931       電腦網路進階      資工二  3.0  張英超   4     2      熱門課程    資工電腦教室
14932        計算機組織      資工二  3.0  陳伯岳   3     5      一般課程     33404
14933       數位系統技術      資工二  2.0  陳仁德   3     1      熱門課程  力行館31102
14934  網際網路資料庫程式設計      資工二  3.0  李志朗   2    10      一般課程  工學院EB211
14935    程式語言理論與實務      資工二  3.0  賴聯福   1     5      一般課程    資工電腦教室
14936      電子技術(一)      資工二  1.0  張家濟   1     3      熱門課程     EB214

[模擬選課] 使用者已選擇：
 ->  可程式邏輯設計 資工二 (週5 第2節起，3.0學分)
 ->  微處理機技術 資工二 (週4 第5節起，2.0學分)

===== Step 2: 推薦核心通識 (已排除衝堂，按歷史熱門度排序) =====
       課程名稱 開課班別(代表)   教師姓名  星期  起始節次    歷史飽和度    歷史中籤率       地點
   博物館與世界文明     核心通識    劉德祥   4     3 9.990000 0.142989     T005
     個人投資理財     核心通識 林逸程吳明政   2     3 7.903333 0.339335    32211
       資訊素養     核心通識    王妙媛   1

輸出中文對齊版

In [11]:
import pandas as pd

# 設定 Pandas 顯示選項，讓中文字能正確對齊
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

def course_recommendation_system(file_path, target_year=114, target_semester=2, target_class='資工三'):
    """
    系級排課與通識推薦系統演算法 (對齊優化版)
    """
    # 1. 讀取資料
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print("錯誤：找不到檔案，請確認檔案路徑是否正確。")
        return

    # --- 預處理：計算歷史核心通識統計數據 ---
    history_mask = (df['學年度'] < target_year) & (df['開課班別(代表)'] == '核心通識')
    ge_history = df[history_mask].groupby('課程名稱').agg({
        '飽和度': 'mean',
        '中籤率': 'mean'
    }).reset_index().rename(columns={'飽和度': '歷史飽和度', '中籤率': '歷史中籤率'})

    # --- Step 1: 列出該系級開設之課程 ---
    dept_courses = df[
        (df['學年度'] == target_year) & 
        (df['學期'] == target_semester) & 
        (df['開課班別(代表)'] == target_class)
    ].copy()

    if dept_courses.empty:
        print(f"找不到 {target_year}-{target_semester} {target_class} 的開課資料。")
        return

    print(f"\n===== Step 1: {target_year}-{target_semester} {target_class} 開課清單 =====")
    # 選擇關鍵欄位顯示
    display_cols = ['課程名稱', '開課班別(代表)', '學分', '教師姓名', '星期', '起始節次', 'cluster標籤', '地點']
    print(dept_courses[display_cols].to_string(index=True))
    
    # --- 模擬使用者挑選 ---
    # 自動選取清單前兩門課作為模擬
    user_selected_indices = dept_courses.index[:2].tolist()
    selected_df = dept_courses.loc[user_selected_indices]
    
    print(f"\n[模擬選課] 使用者已選擇：")
    for _, row in selected_df.iterrows():
        # 使用 f-string 並指定寬度來手動微調對齊
        print(f" -> {row['課程名稱']:<20} (週{row['星期']} 第{row['起始節次']}節起，{row['學分']}學分)")

    # --- Step 2: 動態推薦通識課 (避開衝堂) ---
    ge_candidates = df[
        (df['學年度'] == target_year) & 
        (df['學期'] == target_semester) & 
        (df['開課班別(代表)'] == '核心通識')
    ].copy()
    
    ge_candidates = ge_candidates.merge(ge_history, on='課程名稱', how='left')
    ge_candidates['歷史飽和度'] = ge_candidates['歷史飽和度'].fillna(0)
    ge_candidates['歷史中籤率'] = ge_candidates['歷史中籤率'].fillna(1.0)

    def check_conflict(ge_row, selected_courses_df):
        ge_start = ge_row['起始節次']
        ge_end = ge_start + int(ge_row['學分']) - 1
        for _, sel in selected_courses_df.iterrows():
            if ge_row['星期'] == sel['星期']:
                sel_start = sel['起始節次']
                sel_end = sel_start + int(sel['學分']) - 1
                if max(ge_start, sel_start) <= min(ge_end, sel_end):
                    return True
        return False

    ge_candidates['is_conflict'] = ge_candidates.apply(lambda row: check_conflict(row, selected_df), axis=1)
    recommendations = ge_candidates[ge_candidates['is_conflict'] == False].copy()
    recommendations = recommendations.sort_values(by=['歷史飽和度', '歷史中籤率'], ascending=[False, False])

    print(f"\n===== Step 2: 推薦核心通識 (已排除衝堂，按歷史熱門度排序) =====")
    if not recommendations.empty:
        # 顯示前 10 筆推薦，並縮減欄位名稱以節省空間
        top_10 = recommendations[['課程名稱', '開課班別(代表)', '教師姓名', '星期', '起始節次', '歷史飽和度', '歷史中籤率', '地點']].head(10)
        #top_10.columns = ['課程名稱', '教師', '週', '節', '歷史熱度', '歷史機率', '地點']
        print(top_10.to_string(index=False))
    else:
        print("抱歉，目前時間排程下沒有不衝堂的通識課。")

# 執行
file_name = 'course_cluster3(1).csv'
course_recommendation_system(file_name)


===== Step 1: 114-2 資工三 開課清單 =====
                 課程名稱 開課班別(代表)  學分      教師姓名  星期  起始節次 cluster標籤          地點
14897  資訊科技科教學實習         資工三   2.0        鄭曜忠     1        11    一般課程          T401
14919    系統整合專題(一)         資工三   2.0  易昶霈張家濟     6         1    熱門課程    資工研討室
14920    網路通訊專題(一)         資工三   2.0  詹益禎丁德榮     6         1    熱門課程  資工電腦教室
14921    軟體發展專題(一)         資工三   2.0        陳伯岳     6         1    熱門課程  資工電腦教室
14922        網際網路協定         資工三   3.0        詹益禎     5         2    熱門課程     33401教室
14923            網路安全         資工三   3.0        丁德榮     4         5    熱門課程         34304
14924              機率論         資工三   3.0        施明毅     1         2    熱門課程     33401教室
14925          資料庫系統         資工三   3.0        賴聯福     3         5    熱門課程  資工電腦教室
14926        視窗程式設計         資工三   3.0        黃耀賢     3         2    熱門課程  資工電腦教室
14927              物聯網         資工三   3.0        張英超     2         2    熱門課程  資工電腦教室
14928    UNIX系統程式設計         資工三   3.0          論文     2    